In [ ]:
import os
import pandas as pd 
import numpy as np
from multiprocessing import Pool 
import multiprocessing

import sys
sys.path.append('../../utils/')
from data_loader import data_loader #data_loader.py 파일을 다운 받아 주셔야 합니다. 
from tqdm import tqdm
from functools import partial

import matplotlib.pyplot as plt
from collections import Counter

import glob


In [ ]:
train = pd.read_csv('../../../data/train/train_100.csv')

## 위까지 데이콘 코드 아래부터 추가코드

In [ ]:
train.shape

In [ ]:
train.head(1)

In [ ]:
train.select_dtypes(exclude='object')

In [ ]:
#라벨분포. 라벨 분포에 따라 적은 라벨은 학습이 덜될수도 있을듯?
#업샘플링, 다운샘플링 연구 필요
plt.hist(label.label, bins=50)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
# from sklearn.multioutput import MultiOutputClassifier


In [ ]:
#랜덤포레스트 분류기 사용해서 진행.
#더 좋은 모델 사용하면 오를듯.
#999는 '상태A'임. 다빼버림.

rfc = RandomForestClassifier()

train_X = train[train.label!=999]
train_y = train_X.label

train_X = train_X.drop(['id','time','label'], axis=1)

In [ ]:
train_X.shape, train_y.shape

In [ ]:
#학습

rfc.fit(train_X,train_y)

In [ ]:
# test csv 들을 넣어서 각csv의 row별로 라벨 별 확률값인 predict_proba를 평균내서 각 csv별 predict_proba를 구함.
# 1154, 1168은 문자열포함이라 추론에러남. 그냥 빼고 진행. 나중에 0으로 채움.

results = []

for i in range(828,1548):
    print(i)
    if i != 1154 and i != 1168:
        tmp = pd.read_csv('./test/'+str(i)+'.csv')
        tmp = tmp.drop(['time'], axis=1)
        result = rfc.predict_proba(tmp)

        results.append(np.mean(result,axis=0))

In [ ]:
#위에서 뺀 1154, 1168자리에 0으로 채움

results.insert(326,[0]*198)
results.insert(340,[0]*198)

In [ ]:
#id열 추가하고 DF만듬.

id = pd.DataFrame(np.arange(828,1548), columns=['id'])
submit = pd.DataFrame(results)

submit = pd.concat([id,submit], axis=1)

In [ ]:
submit.head()

In [ ]:
submit.shape


In [ ]:
#id 1154에 0들어갔는지 확인

submit[submit['id']==1154]

In [ ]:
#id 1154에 0들어갔는지 확인

submit[submit['id']==1168]

In [ ]:
#데이터프레임 생성

submit.to_csv('./submit_test.csv', index=False)